In [46]:

print("Ok")

Ok


In [47]:

%pwd

'c:\\Users\\Akash\\Desktop\\499\\OWN\\End-To-End-Medical-Chatbot'

In [3]:
import os
os.chdir("../")

In [4]:

%pwd

'c:\\Users\\Akash\\Desktop\\499\\OWN\\End-To-End-Medical-Chatbot'

In [48]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [49]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [50]:
extracted_data=load_pdf_file(data='Data/')

In [51]:
# extracted_data


In [52]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [53]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [54]:

# text_chunks

In [55]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [56]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [57]:
embeddings = download_hugging_face_embeddings()

In [58]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [59]:

# query_result

In [110]:
from dotenv import load_dotenv
load_dotenv()

True

pcsk_73YUDu_K2qyEkLeH9wGaJd64ykaq8f4LnBzG1SE2RHZWywfNyR3KvZiEsgNoePXYf1ch2i

In [111]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
# OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [62]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [112]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [64]:
from langchain.vectorstores import Pinecone


In [65]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain.vectorstores import Pinecone
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [81]:
# Load Existing index 

from langchain.vectorstores import Pinecone

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [113]:
docsearch

In [114]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [115]:

retrieved_docs = retriever.invoke("What is Acne?")

In [116]:

retrieved_docs

[Document(metadata={'page': 39.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(metadata={'page': 38.0, 'source': 'Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 37.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term fo

In [117]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0.4, max_tokens=500)

In [118]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [119]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:

response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a common skin disease, characterized by pimples on the face, chest, and back, resulting from clogged pores with oil, dead skin cells, and bacteria. Acne vulgaris, its medical term, affects nearly 17 million people in the United States.


In [121]:

response = rag_chain.invoke({"input": "what is stats?"})
print(response["answer"])

The question asks for statistics related to Sudden Unexpected Death (SUD) in athletes. The passage mentions that the yearly rate for occurrence of SUD in people less than 35 years of age is less than seven incidents per 100,000, and of all SUD cases, only about 8% are exercise related. Therefore, each year approximately 25 athletes experience SUD in the United States.
